In [10]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

6933dbce-9daf-4d2e-b6f9-78516155c17a


In [11]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '6d490d2d04897c513f22e5f37b27d988', 'Date': 'Wed, 21 Aug 2024 08:24:04 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [16]:
import json
data = json.load(open("reviews_updated.json"))
data['reviews']

[{'professor': 'Dr. Emily Chen',
  'subject': 'Physics',
  'stars': 5,
  'review': "Dr. Chen's lectures are incredibly engaging. She explains complex concepts with clarity and enthusiasm."},
 {'professor': 'Prof. Michael Johnson',
  'subject': 'History',
  'stars': 4,
  'review': 'Very knowledgeable about the subject. Sometimes goes off on tangents, but overall a good instructor.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Biology',
  'stars': 3,
  'review': 'Lectures can be dry, but Dr. Thompson is always willing to help during office hours.'},
 {'professor': 'Prof. David Lee',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Excellent teacher! Makes calculus understandable and even enjoyable.'},
 {'professor': 'Dr. Rachel Green',
  'subject': 'Psychology',
  'stars': 4,
  'review': 'Engaging lectures and interesting assignments. Could improve on providing timely feedback.'},
 {'professor': 'Prof. Robert Brown',
  'subject': 'Chemistry',
  'stars': 2,
  'review': 'Diff

In [17]:
processed_data = []
client = OpenAI() 

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [21]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 25}

In [20]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 22}},
 'total_vector_count': 22}